In [13]:
# imports
from bs4 import BeautifulSoup
import urllib.request
import csv

import pandas as pd

In [14]:
# url
url = 'https://www.fasttrack.co.uk/league-tables/tech-track-100/league-table/'

# query
page = urllib.request.urlopen(url)

# parsing with bs
soup = BeautifulSoup(page, 'html.parser')

In [15]:
#print(soup)

In [16]:
# results within the table
table = soup.find('table', attrs={'class': 'tableSorter'})
results = table.find_all('tr')

In [17]:
print(len(results))

101


In [18]:
# creating headers
rows = []
rows.append(['Rank', 'Company', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 
             'Latest sales £000s', 'Staff', 'Comments'])

In [19]:
print(rows)

[['Rank', 'Company', 'Webpage', 'Description', 'Location', 'Year end', 'Annual sales rise over 3 years', 'Latest sales £000s', 'Staff', 'Comments']]


In [20]:
# looping trough results 
for result in results:
    data = result.find_all('td')
    if len(data) == 0: 
        continue
        
    # writing columns to variables
    rank = data[0].getText()
    company = data[1].getText()
    location = data[2].getText()
    yearEnd = data[3].getText()
    salesRise = data[4].getText()
    sales = data[5].getText()
    staff = data[6].getText()
    comments = data[7].getText()
    
    # further cleaning of the data

    # separating description from company name
    companyname = data[1].find('span', attrs={'class':'company-name'}).getText()
    description = company.replace(companyname, '')
    company = company.replace(description, '')

    # removing unwanted symbols from sales
    sales = sales.strip('*').strip('†').replace(',','')

    # extracting company website
    url = data[1].find('a').get('href')
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, 'html.parser')

    # try/catch where websites are not displayed
    try:
        tableRow = soup.find('table').findAll('tr')[-1]
        webpage = tableRow.find('a').get('href')
    except:
        webpage = None
        
    # adding the cleaned data to the array rows
    rows.append(([rank, company, webpage, description, location, yearEnd, salesRise, sales, staff, comments]))

In [21]:
#print(rows)

In [22]:
# writing the rows into a csv file
with open('techtrack100.csv', 'w', newline = '') as f_output:
    csv_output = csv.writer(f_output)
    csv_output.writerows(rows)

In [23]:
techtrack100 = pd.read_csv("techtrack100.csv", index_col = 0)

In [24]:
techtrack100.head(5)

,Company,Webpage,Description,Location,Year end,Annual sales rise over 3 years,Latest sales £000s,Staff,Comments
Rank,,,,,,,,,
1,Plan.com,http://www.plan.com,Communications provider,Isle of Man,Sep-17,364.38%,35418,90,About 650 partners use its telecoms platform t...
2,PsiOxus,http://www.psioxus.com,Biotechnology developer,Oxfordshire,Dec-17,311.67%,53136,54,Received a $15m milestone payment from its dev...
3,CensorNet,http://www.censornet.com,Cloud security software developer,Basingstoke,Dec-17,210.17%,7535,77,"Has more than 4,000 customers, including McDon..."
4,thoughtonomy,http://www.thoughtonomy.com,Automation software developer,East London,May-18,205.20%,16916,100,It sells to 28 countries and 50% of revenue is...
5,Perkbox,http://www.perkbox.com,Employee engagement services,Central London,Dec-17,204.12%,34700,200,Acquired software platform Loyalty Bay for an ...
